In [0]:
import dlt
from pyspark.sql import functions as F
from config.config import *

In [0]:
# Use DLT to create a silver table which merges the data from bt_claims_batch and bt_claims_stream
# Remove null data

@dlt.table(
    name=st_fact_claims_merged,
    comment="Merges the data from bt_claims_batch and bt_claims_stream",
    table_properties={"pipelines.reset.allowed": "true"}
)
@dlt.expect_or_drop(
    "not_null_keys",
    "ClaimID IS NOT NULL AND MemberID IS NOT NULL AND ProviderID IS NOT NULL"
)
def silver_claims_combined():
    cols_relevant = [
        "ClaimID",
        "MemberID",
        "ProviderID",
        "ClaimDate",
        "Amount",
        "Status",
        "ICD10Codes",
        "CPTCodes",
        "meta_timestamp",
        "meta_source_file",
    ]

    df_claims_batch = spark.read.table(bt_fact_claims_batch).select(cols_relevant)
    df_claims_stream = spark.read.table(bt_fact_claims_stream).select(cols_relevant)


    return df_claims_batch.unionByName(df_claims_stream)